<a href="https://colab.research.google.com/github/harsh16kh/NLP/blob/main/Copy_of_20IE10049_Assn2_NLP_A21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [ ]:
import re
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [ ]:
#Load the IMDB dataset. You can load it using pandas as dataframe
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#Read the dataset
df = pd.read_csv('drive/MyDrive/IMDB_dataset/IMDB Dataset.csv')

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [ ]:
import nltk, math
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def apply1(text):
    text = text.replace('<br />', ' ')
    text = re.sub('http\S+',' ',text)
    return text

L=[]
def apply2(text):
    for sent in sent_tokenize(text):
        L.append(len(word_tokenize(sent)))
    return text

def apply3(text):
    text = re.sub(r'[^a-zA-Z0-9 ]',' ',text)
    text = text.lower()
    words = word_tokenize(text)
    filtered_words = list(set(words)-set(stopwords.words('english')))
    text = ' '.join(filtered_words)
    return text
    



In [ ]:
df['review']=df['review'].apply(apply1)
df['review']=df['review'].apply(apply2)
average = sum(L)/len(L)
df['review']=df['review'].apply(apply3)

In [ ]:
#Stemming
from nltk.stem import PorterStemmer, WordNetLemmatizer
ps = PorterStemmer()
le = WordNetLemmatizer()
def lemmatization(text):
    words = word_tokenize(text)
    for i in range(len(words)):
        words[i] = le.lemmatize(words[i])
    text = words[0]
    for i in range(1,len(words)):
        text += ' '
        text += words[i]
    return text

def stemming(text):
    words = word_tokenize(text)
    for i in range(len(words)):
        words[i] = ps.stem(words[i])
    text = words[0]
    for i in range(1,len(words)):
        text += ' '
        text += words[i]
    return text

df['review'] = df['review'].apply(stemming)
df['review'] = df['review'].apply(lemmatization)

In [ ]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
print('Average length of sentence: ', sum(L)/len(L))

Average length of sentence:  21.454168404279315


In [ ]:
df.head()

,review,sentiment
0,due becom one death punch first got happen acc...,positive
1,realli one got mural surfac actor refer littl ...,positive
2,point one manag air decad averag crown right w...,positive
3,divorc life well becom watch instead expect th...,negative
4,one point habitat connect talent await situat ...,positive


In [ ]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']
reviews,labels

(0        due becom one death punch first got happen acc...
 1        realli one got mural surfac actor refer littl ...
 2        point one manag air decad averag crown right w...
 3        divorc life well becom watch instead expect th...
 4        one point habitat connect talent await situat ...
                                ...                        
 49995    soul life good becom one yr first 20 catwoman ...
 49996    springtim sing stuck soundtrack beyond thousan...
 49997    two author first wrong religion movi eye catho...
 49998    one mumbl appropri explain scene park progress...
 49999    good mccoy hardli well worst one spock chanc g...
 Name: review, Length: 50000, dtype: object, 0        positive
 1        positive
 2        positive
 3        negative
 4        positive
            ...   
 49995    positive
 49996    negative
 49997    negative
 49998    negative
 49999    negative
 Name: sentiment, Length: 50000, dtype: object)

In [ ]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print(encoder.classes_)

['negative' 'positive']


In [ ]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2,random_state=42)
# train_sentences, test_sentences, train_labels, test_labels

# *LSTM* based Classifier

Use the above train and test splits.

In [ ]:
# Hyperparameters of the model
vocab_size = 3000# choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = max([len(text) for text in reviews])# choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5722, 100)         300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 2 #reduced the number of epochs due to time constraints. The runtime in google colab gets disconnected after a certain point of time.
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/2
1125/1125 [==============================] - 1329s 1s/step - loss: 0.3756 - accuracy: 0.8344 - val_loss: 0.3104 - val_accuracy: 0.8720
Epoch 2/2
1125/1125 [==============================] - 1328s 1s/step - loss: 0.2772 - accuracy: 0.8882 - val_loss: 0.3077 - val_accuracy: 0.8773


In [ ]:
# Calculate accuracy on Test data
prediction = model.predict(test_padded)

# Get probabilities
print('Probabilities: ', prediction)



Probabilities:  [[0.40844065]
 [0.8989985 ]
 [0.49020112]
 ...
 [0.90914536]
 [0.08338819]
 [0.60208064]]


In [ ]:
# Get labels based on probability 1 if p>= 0.5 else 0
labels = np.zeros((10000,1))
for i in range(10000):
  if prediction[i][0] > 0.5:
    labels[i][0]=1
c = 0
for i in range(10000):
  if labels[i] == test_labels[i]:
    c += 1


# Accuracy : one can use classification_report from sklearn
print('Accuracy: ', c/100, '%')

Accuracy:  87.14 %


## Get predictions for random examples

In [ ]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
p = model.predict(padded)
print(p)

# Get labels based on probability 1 if p>= 0.5 else 0
for i in range(3):
  if p[i]>0.5:
    print(sentence[i], ': Positive')
  else:
    print(sentence[i], ': Negative')


[[0.49791795]
 [0.57001346]
 [0.43192655]]
The movie was very touching and heart whelming : Negative
I have never seen a terrible movie like this : Positive
the movie plot is terrible but it had good acting : Negative
